## Imports

Importing the required modules for the project.

The `OpenAI` client class from the `openai` API for interacting with ChatGPT, and the `ChatCompletion` type for typing in various functions.

In [740]:
from openai import OpenAI
from openai.types.chat.chat_completion import ChatCompletion

The ```dotenv_values``` function from the `dotenv` module for accessing configuration file properties.

In [741]:
from dotenv import dotenv_values

Numpy and Pandas for advanced array and dataframe functionality, respectively.

In [742]:
import numpy as np
import pandas as pd

The iterable type for typing in various functions.

In [743]:
from collections.abc import Iterable

## Global Settings

Global settings and variables to be used throughout the project.

Creating the OpenAI client and setting the OpenAI API key.

**NOTE:** You should have your key stored in a file named `.env` inside the parent folder of this notebook. 

In this file, you need to store your key as such:

`OPENAI_API_KEY={YOUR_API_KEY}`

In [744]:
openai_client = OpenAI(
    api_key=dotenv_values("../../.env")["OPENAI_API_KEY"]
    )

Defining a random number generator to be used throughout the project in order to ensure the same random results are obtained each time the project is run.

In [745]:
rng = np.random.default_rng(seed=101)

## Function Definitions

Defining the public and private functions to be used.

### Private Functions

These are private, helper functions. They are not meant to be used on their own, and are instead called by the main functions to be utilised by the user.

In [746]:
def document_class_instance(
        instance: any,
        indent: int = 0
        ) -> None:
    """
    Recursively documents the properties of the given object instance,
    including nested properties, down to the primitive types.

    Parameters:
        instance (any): The object instance to document.
        indent (int): The current level of indentation for formatting.

    Returns:
        None
    """

    indentation = "    " * indent
    print(f"{indentation}Instance Class: {type(instance)}")

    if hasattr(instance, '__dict__'):
        for key, value in vars(instance).items():
            print(f"{indentation}{key} (type: {type(value)}):")
            if hasattr(value, '__dict__'):
                document_class_instance(value, indent + 1)
            elif isinstance(value, (list, tuple)):
                print(f"{indentation}    {key} (list or tuple):")
                for idx, item in enumerate(value):
                    print(f"{indentation}        [{idx}] (type: {type(item)}):")
                    if hasattr(item, '__dict__'):
                        document_class_instance(item, indent + 2)
                    else:
                        print(f"{indentation}        {item}")
            elif isinstance(value, dict):
                print(f"{indentation}    {key} (dictionary):")
                for dict_key, dict_value in value.items():
                    print(f"{indentation}        {dict_key} (type: {type(dict_value)}):")
                    if hasattr(dict_value, '__dict__'):
                        document_class_instance(dict_value, indent + 2)
                    else:
                        print(f"{indentation}        {dict_value}")
            else:
                print(f"{indentation}    {value}")
    else:
        print(f"{indentation}{instance}")

    print()  # Empty line for spacing


In [747]:
def flatten_dict(
        dictionary: dict,
        parent_key: str = '',
        sep: str = '_',
        counter: dict = None
        ) -> dict:

    if counter is None:
        counter = {}
    
    items = []

    for key, value in dictionary.items():
        new_key = f"{parent_key}{sep}{key}" if parent_key else key

        if isinstance(value, dict):
            items.extend(
                flatten_dict(
                    value,
                    new_key,
                    sep=sep,
                    counter=counter
                ).items()
            )
            
        elif isinstance(value, Iterable) and not isinstance(value, str):
            for index, item in enumerate(value):
                if isinstance(item, dict):
                    items.extend(
                        flatten_dict(
                            item,
                            f"{new_key}_{index}",
                            sep=sep,
                            counter=counter
                        ).items()
                    )
                else:
                    items.append((f"{new_key}_{index}", item))
        else:
            if new_key in counter:
                counter[new_key] += 1
                new_key = f"{new_key}_{counter[new_key]}"
            else:
                counter[new_key] = 1
                
            items.append((new_key, value))
            
    return dict(items)

In [748]:
def combine_chat_completions(responses: list[ChatCompletion]):

    responses_as_dfs: list[pd.DataFrame] = []

    for chat_completion in responses:
        chat_dict = chat_completion.to_dict()
        flattened_chat_dict = flatten_dict(chat_dict)

        filtered_dict = {
            key: value
            for key, value
            in flattened_chat_dict.items() if "_bytes_" not in key
        }

        response_as_df = pd.DataFrame(
            filtered_dict,
            index=[0]
            )
        
        responses_as_dfs.append(response_as_df)

    df = pd.concat(
        responses_as_dfs,
        ignore_index=True
        )
    
    return df

In [749]:
def create_initial_messages(system_message: str) -> list[dict[str, str]]:
    messages: list[dict[str, str]] = [
        {"role": "system", "content": system_message}
    ]

    return messages

### Public Functions

The below are the public functions to be used in this project.

In [750]:
def get_chatgpt_response(
        *args, 
        messages: list[dict[str, str]],
        engine: str = "gpt-3.5-turbo",
        max_tokens: int = 500,
        temperature: float = 0.0,
        n: int = 1,
        logprobs: bool = False,
        top_logprobs: int = 10,
        **kwargs
        ):

    response = openai_client.chat.completions.create(
        messages=messages,
        model=engine,
        max_tokens=max_tokens,
        temperature=temperature,
        n=n,
        seed=42,
        logprobs=logprobs,
        top_logprobs=top_logprobs
    )

    return response

In [751]:
def get_chatgpt_responses(
        *args,
        system_message: str,
        prompts: list[str],
        engine: str = "gpt-3.5-turbo",
        max_tokens: int = 500,
        temperature: float = 0.0,
        n: int = 1,
        logprobs: bool = False,
        top_logprobs: int = 10,
        **kwargs
):
    
    if len(prompts) > 1:
        n = 1

    chat: dict[str, list] = {
        "messages": create_initial_messages(system_message),
        "responses": []
              }

    for prompt in prompts:
        
        chat["messages"].append({"role": "user", "content": prompt})

        response = get_chatgpt_response(
            messages=chat["messages"],
            engine=engine,
            max_tokens=max_tokens,
            temperature=temperature,
            n=n,
            logprobs=logprobs,
            top_logprobs=top_logprobs
            )
        
        response_message_content: str = response.choices[0].message.content

        chat["responses"].append(response)
        chat["messages"].append({"role": "assistant", "content": response_message_content})

        print(f"\tgpt: {chat["messages"][-1]["content"]}")

    return chat

In [752]:
def run_scenario(
        scenario: pd.DataFrame,
        n: int = 1,
        engine: str = "gpt-3.5-turbo",
        max_tokens: int = 500,
        temperature: float = 0.0,
        logprobs: bool = False,
        top_logprobs: int = 10,
        with_options: bool = False
        ):
    
    scenario = scenario[scenario["question_type"] == "Closed-ended"]

    system_message = scenario.iloc[0]["system_message_content"]
    system_message += "\n\nRespond only with a single digit." if (scenario["story_language"] == "English").all() else "Sadece tek bir rakam ile yanıt ver."

    prompts = scenario["question_content"].to_list()

    prompts[0] = f"{scenario.iloc[0]["story_content"]}\n\n{prompts[0]}"

    if with_options == True:
        options = [item.split(", ") if isinstance(item, str) else None for item in scenario["question_options"].to_list()]
        for index, (prompt, option_list) in enumerate(zip(prompts, options)):
            if option_list == None:
                continue
            option_str = "\n".join(f"{i + 1} {element}" for i, element in enumerate(option_list))
            respond_prompt = ""
            if {"Evet", "Hayır", "Yes", "No", "Rebecca's mother's gift", "neighbour's cat", "Melis'in annesinin hediyesi", "komşunun kedisi"} & set(option_list):
                respond_prompt = "Respond with the digit corresponding to the correct answer." if (scenario["story_language"] == "English").all() else "Doğru cevaba denk gelen rakam ile yanıt ver."
            else:
                respond_prompt = "Respond with the digit corresponding to the correct answer to replace {RESPONSE}." if (scenario["story_language"] == "English").all() else "{CEVAP} yerine gelecek doğru cevaba denk gelen rakam ile yanıt ver."
            prompts[index] = f"{prompt}\n\n{option_str}\n\n{respond_prompt}"

    
    responses = get_chatgpt_responses(
        system_message=system_message,
        prompts=prompts,
        engine=engine,
        max_tokens=max_tokens,
        temperature=temperature,
        n=n,
        logprobs=logprobs,
        top_logprobs=top_logprobs
        )
    
    return responses

## Variable Definitions

Creating the required variables for our project.

Creating the questions dictionary in order to test for four different variations of the question to be asked. Two of these questions are "observation" based, while the remaining two are "intervention" based.

## Examples

We will have a look at an example prompt and response.

In [753]:
df = pd.read_csv(
    "../input/parameters_expanded/03_parameters_expanded.csv",
    index_col=0
    )

In [754]:
df.head(3)

,study_id,study_name,scenario_id,scenario_code,story_id,chat_id,story_common_id,story_category,story_name,story_content,...,question_content,question_options,question_type,question_language,question_has_fbv_zan,question_has_fbv_san,question_tom_order,question_tom_type,answer_id,answer_correct
0,1,Ullman Replication,1,1-EN/1,1-EN,1,1,Unexpected Contents,Base 1,Here is a bag filled with popcorn. There is no...,...,She believes that the bag is full of {RESPONSE}.,"popcorn, chocolate",Closed-ended,English,NaN,NaN,1,Belief,1,chocolate
1,1,Ullman Replication,2,1A-EN/1,1A-EN,1,1A,Unexpected Contents,Transparent,Here is a bag filled with popcorn. There is no...,...,She believes that the bag is full of {RESPONSE}.,"popcorn, chocolate",Closed-ended,English,NaN,NaN,1,Belief,11,popcorn
8,1,Ullman Replication,3,1B-EN/66,1B-EN,66,1B,Unexpected Contents,Uninformative,Here is a bag filled with popcorn. There is no...,...,She believes that the bag is full of {RESPONSE}.,"popcorn, chocolate, uncertainty",Closed-ended,English,NaN,NaN,1,Belief,21,uncertainty


In [755]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 327 entries, 0 to 322
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   study_id                 327 non-null    int64  
 1   study_name               327 non-null    object 
 2   scenario_id              327 non-null    int64  
 3   scenario_code            327 non-null    object 
 4   story_id                 327 non-null    object 
 5   chat_id                  327 non-null    int64  
 6   story_common_id          327 non-null    object 
 7   story_category           327 non-null    object 
 8   story_name               327 non-null    object 
 9   story_content            327 non-null    object 
 10  story_language           327 non-null    object 
 11  chat_name                327 non-null    object 
 12  chat_language            327 non-null    object 
 13  chat_has_fbv_zan         77 non-null     float64
 14  chat_has_fbv_san         76 non

In [756]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
study_id,327.0,2.146789,0.702655,1.0,2.0,2.0,3.0,3.0
scenario_id,327.0,74.452599,27.659355,1.0,58.5,79.0,92.0,116.0
chat_id,327.0,37.311927,18.578319,1.0,23.0,37.0,50.0,76.0
chat_has_fbv_zan,77.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
chat_has_fbv_san,76.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
system_message_id,327.0,4.027523,1.497188,1.0,3.0,4.0,5.0,6.0
question_index,327.0,1.691131,1.733648,0.0,0.0,1.0,3.0,7.0
question_common_id,327.0,10.055046,9.704058,1.0,3.0,6.0,15.0,35.0
question_has_fbv_zan,41.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
question_has_fbv_san,41.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0


In [757]:
counter = 0

In [758]:
df[df["scenario_id"] == 51]

,study_id,study_name,scenario_id,scenario_code,story_id,chat_id,story_common_id,story_category,story_name,story_content,...,question_content,question_options,question_type,question_language,question_has_fbv_zan,question_has_fbv_san,question_tom_order,question_tom_type,answer_id,answer_correct
4,2,Ullman Expansion,51,1-EN/21,1-EN,21,1,Unexpected Contents,Base 1,Here is a bag filled with popcorn. There is no...,...,She believes that the bag is full of {RESPONSE}.,"popcorn, chocolate",Closed-ended,English,NaN,NaN,1,Belief,1,chocolate
34,2,Ullman Expansion,51,1-EN/21,1-EN,21,1,Unexpected Contents,Base 1,Here is a bag filled with popcorn. There is no...,...,She is delighted to have found this bag. She l...,"popcorn, chocolate",Closed-ended,English,NaN,NaN,1,Action,2,chocolate
172,2,Ullman Expansion,51,1-EN/21,1-EN,21,1,Unexpected Contents,Base 1,Here is a bag filled with popcorn. There is no...,...,Why does she think that the bag is full of tha...,NaN,Open-ended,English,NaN,NaN,1,Explanation,3,Because the label says there is chocolate in t...
228,2,Ullman Expansion,51,1-EN/21,1-EN,21,1,Unexpected Contents,Base 1,Here is a bag filled with popcorn. There is no...,...,She opens the bag and looks inside. She can cl...,"popcorn, chocolate",Closed-ended,English,NaN,NaN,0,Reality,4,popcorn


In [759]:
top_logprob_tokens = ['choices_0_logprobs_content_0_top_logprobs_0_token',
 'choices_0_logprobs_content_0_top_logprobs_1_token',
 'choices_0_logprobs_content_0_top_logprobs_2_token',
 'choices_0_logprobs_content_0_top_logprobs_3_token',
 'choices_0_logprobs_content_0_top_logprobs_4_token',
 'choices_0_logprobs_content_0_top_logprobs_5_token',
 'choices_0_logprobs_content_0_top_logprobs_6_token',
 'choices_0_logprobs_content_0_top_logprobs_7_token',
 'choices_0_logprobs_content_0_top_logprobs_8_token',
 'choices_0_logprobs_content_0_top_logprobs_9_token',
 'choices_0_logprobs_content_0_top_logprobs_10_token',
 'choices_0_logprobs_content_0_top_logprobs_11_token',
 'choices_0_logprobs_content_0_top_logprobs_12_token',
 'choices_0_logprobs_content_0_top_logprobs_13_token',
 'choices_0_logprobs_content_0_top_logprobs_14_token',
 'choices_0_logprobs_content_0_top_logprobs_15_token',
 'choices_0_logprobs_content_0_top_logprobs_16_token',
 'choices_0_logprobs_content_0_top_logprobs_17_token',
 'choices_0_logprobs_content_0_top_logprobs_18_token',
 'choices_0_logprobs_content_0_top_logprobs_19_token']

The cell below is the only part which you need to modify. Change `engine="engine_name"` to the engine you wish to use. Modify the other parameters as you see fit as well. If `with_options=True`, then only closed-ended questions will be sent, receiving responses in the format of a digit for the chosen option.

In [760]:
output = []
#df["scenario_id"].unique()
for scenario_id in df["scenario_id"].unique():
    print(f"scenario_id: {scenario_id}")
    scenario = df[df["scenario_id"] == scenario_id].reset_index(drop=True)
    # if not (scenario["story_common_id"] == "3A").all():
    #     continue

    concat = None

    scenario = scenario[scenario["question_type"] == "Closed-ended"]

    while True:
        responses = run_scenario(
            scenario,
            engine="gpt-4-turbo",
            logprobs=True,
            top_logprobs=20,
            max_tokens=1,
            with_options=True
            )
        
        responses_df = combine_chat_completions(
            responses["responses"]).reset_index(drop=True)

        concat = pd.concat([scenario.reset_index(drop=True), responses_df.reset_index(drop=True)], axis=1).reset_index(drop=True).dropna(subset=["scenario_id"])
        # if all(concat.apply(lambda row: {str(x) for x in range(1, row["question_options"].count(", ") + 2)}.issubset(set(row[top_logprob_tokens])), axis=1)):
        #     break
        # if any(concat[top_logprob_tokens].apply(pd.to_numeric, errors="coerce").notnull()):
        #     break
        if pd.to_numeric(concat["choices_0_message_content"], errors="coerce").notnull().all():
            break
        else:
            display(scenario)
            display(responses_df["choices_0_message_content"])
            display(concat)

    concat["session_counter"] = counter
    counter += 1
    output.append(concat)

scenario_id: 1
	gpt: 2
scenario_id: 2
	gpt: 2
scenario_id: 3
	gpt: 3
scenario_id: 4
	gpt: 1
scenario_id: 5
	gpt: 1
scenario_id: 6
	gpt: 2
scenario_id: 7
	gpt: 2
scenario_id: 8


KeyboardInterrupt: 

In [769]:
output[0][top_logprob_tokens]

,choices_0_logprobs_content_0_top_logprobs_0_token,choices_0_logprobs_content_0_top_logprobs_1_token,choices_0_logprobs_content_0_top_logprobs_2_token,choices_0_logprobs_content_0_top_logprobs_3_token,choices_0_logprobs_content_0_top_logprobs_4_token,choices_0_logprobs_content_0_top_logprobs_5_token,choices_0_logprobs_content_0_top_logprobs_6_token,choices_0_logprobs_content_0_top_logprobs_7_token,choices_0_logprobs_content_0_top_logprobs_8_token,choices_0_logprobs_content_0_top_logprobs_9_token,choices_0_logprobs_content_0_top_logprobs_10_token,choices_0_logprobs_content_0_top_logprobs_11_token,choices_0_logprobs_content_0_top_logprobs_12_token,choices_0_logprobs_content_0_top_logprobs_13_token,choices_0_logprobs_content_0_top_logprobs_14_token,choices_0_logprobs_content_0_top_logprobs_15_token,choices_0_logprobs_content_0_top_logprobs_16_token,choices_0_logprobs_content_0_top_logprobs_17_token,choices_0_logprobs_content_0_top_logprobs_18_token,choices_0_logprobs_content_0_top_logprobs_19_token
0,2,**,Based,Since,Given,1,Sam,\n,,Respond,The,２,Considering,`,​,3,If,She,\[,Despite


In [ ]:
len(output)

116

In [ ]:
df = pd.concat(
    output,
    axis=0,
    ignore_index=True
    ).reset_index(drop=True)

In [ ]:
df.to_csv("../output/responses_raw/05_20_options_gpt_4.csv")